In [ ]:
model_name = 'SGDClassifier'
model_variant = 'base'

In [3]:
import pandas as pd

df = pd.read_csv('spotify_dataset.csv')
df = df.drop(columns=['Song ID', 'Artist', 'Number of Times Charted',
 'Week of Highest Charting', 'Song Name', 'Streams', 'Release Date',
  'Weeks Charted', 'Popularity', 'Index'])
df = df.replace(' ','0')

print('Shape of dataframe is:', df.shape)

df.head(5)

Shape of dataframe is: (1556, 13)


,Highest Charting Position,Artist Followers,Genre,Danceability,Energy,Loudness,Speechiness,Acousticness,Liveness,Tempo,Duration (ms),Valence,Chord
0,1,3377762,"['indie rock italiano', 'italian pop']",0.714,0.8,-4.808,0.0504,0.127,0.359,134.002,211560,0.589,B
1,2,2230022,['australian hip hop'],0.591,0.764,-5.484,0.0483,0.0383,0.103,169.928,141806,0.478,C#/Db
2,1,6266514,['pop'],0.563,0.664,-5.044,0.154,0.335,0.0849,166.928,178147,0.688,A
3,3,83293380,"['pop', 'uk pop']",0.808,0.897,-3.712,0.0348,0.0469,0.364,126.026,231041,0.591,B
4,5,5473565,"['lgbtq+ hip hop', 'pop rap']",0.736,0.704,-7.409,0.0615,0.0203,0.0501,149.995,212000,0.894,D#/Eb


In [4]:
X = df.drop(columns=['Highest Charting Position'])
y = df['Highest Charting Position'].values

X['Artist Followers'] = X['Artist Followers'].astype(int)
X['Danceability'] = X['Danceability'].astype(float)
X['Energy'] = X['Energy'].astype(float)
X['Loudness'] = X['Loudness'].astype(float)
X['Speechiness'] = X['Speechiness'].astype(float)
X['Acousticness'] = X['Acousticness'].astype(float)
X['Liveness'] = X['Liveness'].astype(float)
X['Tempo'] = X['Tempo'].astype(float)
X['Duration (ms)'] = X['Duration (ms)'].astype(int)
X['Valence'] = X['Valence'].astype(float)

split = round(len(df)*0.6)
X_train = df[:split]
y_train = y[:split]
X_test = df[split:]
y_test = y[split:]

print('Shape of X_train is:', X_train.shape)
print('Shape of y_train is:', y_train.shape)
print('Shape of X_test is:', X_test.shape)
print('Shape of y_test is:', y_test.shape)



Shape of X_train is: (934, 13)
Shape of y_train is: (934,)
Shape of X_test is: (622, 13)
Shape of y_test is: (622,)


In [5]:
# SETTING UP KERAS CALLBACKS

import os
import time
import tensorflow as tf
from keras import callbacks

# Borrowed from: https://www.geeksforgeeks.org/choose-optimal-number-of-epochs-to-train-a-neural-network-in-keras/
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 10, 
                                        restore_best_weights = True)

model_name = 'entire_sentences_multi_label'
model_variant = 'base'

def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        exp = lr0 * 0.1**(epoch / s)
        tf.summary.scalar('learning rate', data=exp, step=epoch)
        return exp
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=10)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

root_logdir = os.path.join(os.curdir, "./logged_models/" + model_name + '/' + model_variant)
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_log_dir = get_run_logdir()
file_writer = tf.summary.create_file_writer(run_log_dir + "/metrics")
file_writer.set_as_default()

tensorboard_cb = tf.keras.callbacks.TensorBoard(run_log_dir)

my_callbacks = [earlystopping, lr_scheduler, tensorboard_cb]

In [15]:
from tensorflow.keras import layers

model = tf.keras.Sequential(
    
)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

history = model.fit(X_train, y_train, epochs=50, callbacks=my_callbacks, validation_split=0.2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).